<a href="https://colab.research.google.com/github/htkim27/sklearn/blob/master/code/ML/regularized_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!git clone https://github.com/htkim27/sklearn.git
!git clone https://github.com/youngsilpark/Export_Prediction.
!pip install statsmodels

import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
import itertools
from tqdm import tqdm


fatal: destination path 'sklearn' already exists and is not an empty directory.
Cloning into 'Export_Prediction.'...
fatal: could not read Username for 'https://github.com': No such device or address
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
filename= '/content/drive/MyDrive/Export_Prediction-master/data/added_data.csv'
data = pd.read_csv(filename)
data = data.iloc[:-6,1:] # 202107-12 제거
data_train = data.iloc[:-6,:]
data_test = data.iloc[-6:,:] # 202101-06
X_train = data_train.iloc[:,1:]
X_test = data_test.iloc[:,1:]
y_train = data_train.iloc[:,0]
y_test = data_test.iloc[:,0]

(6, 9)


#ARIMA로 독립변수 추출

In [17]:
filename='/content/drive/MyDrive/Export_Prediction-master/data/added_data.csv'
data = pd.read_csv(filename)
data = data.iloc[:-6,1:]  #202107-12 제거
data.head()
data['date']=pd.to_datetime(data['date'])
print(data.columns)

Index(['date', 'korea', 'china', 'USA', 'vietnam', 'hongkong', 'japan', 'EU',
       'SSEC', 'SP500', 'USD_KRW'],
      dtype='object')


In [27]:
forcast_dict = {}
best_params = {}

for i in range(2,11):
  column = str(data.columns[i])
  timeSeries = data.loc[:,['date',column]]
  timeSeries.index = timeSeries.date
  ts = timeSeries.drop('date',axis=1)
  ts = ts.iloc[:-6,:]

###################
  p = range(0,3)
  d = range(1,2)
  q = range(0,6)

  pdq = list(itertools.product(p,d,q))

  aic = []
  params = []

  with tqdm(total = len(pdq)) as pg:
      for i in pdq:
          pg.update(1)
          try:
              model = ARIMA(data[column], order=(i))
              model_fit = model.fit()
              aic.append(round(model_fit.aic,2))
              params.append((i))
          except:
              continue

#####################
  optimal = [(params[i],j) for i,j in enumerate(aic) if j == min(aic)]
  model = ARIMA(ts, order = optimal[0][0])
  model_fit = model.fit()
  forecast212 = model_fit.predict('2021-01-01','2021-06-01', typ='levels')
  forcast_dict[column] = list(forecast212)
  best_params[column] = list(optimal[0][0])

print(forcast_dict)
print(best_params)

100%|██████████| 18/18 [00:04<00:00,  4.08it/s]
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)
 39%|███▉      | 7/18 [00:00<00:00, 66.88it/s]/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
100%|██████████| 18/18 [00:03<00:00,  4.53it/s]
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:16

{'china': [12341723020.815351, 12132232348.473543, 12225030860.675817, 12362068983.06136, 12417211756.704187, 12399742106.773584], 'USA': [6752979530.172898, 6534090367.760596, 6854584791.815634, 6719012891.651906, 6658830817.767772, 6901676664.092443], 'vietnam': [4971353872.7561035, 4894430203.856111, 4882331030.436537, 4891899945.797018, 4908711587.350752, 4927944166.12894], 'hongkong': [2607877591.4383435, 2655222137.891178, 3046825094.553913, 2626122848.1815753, 2661974524.5381393, 3074466198.492547], 'japan': [2293008996.6371803, 2194851072.1353636, 2286999531.053775, 2201260380.4168396, 2293028090.1750817, 2207668873.74598], 'EU': [4488089374.899037, 4533165590.329529, 4547960510.477911, 4532673735.81602, 4551947477.160903, 4590099868.215004], 'SSEC': [3371.1043275254465, 3375.093724152617, 3378.2242469204707, 3380.418431034073, 3390.012806840532, 3398.1515495020453], 'SP500': [3721.4151087633927, 3730.4967735869745, 3739.5784384105564, 3748.660103234138, 3757.74176805772, 3766.

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


In [21]:
type(optimal[0][0])

tuple

In [28]:
forcast_dict

{'EU': [4488089374.899037,
  4533165590.329529,
  4547960510.477911,
  4532673735.81602,
  4551947477.160903,
  4590099868.215004],
 'SP500': [3721.4151087633927,
  3730.4967735869745,
  3739.5784384105564,
  3748.660103234138,
  3757.74176805772,
  3766.823432881302],
 'SSEC': [3371.1043275254465,
  3375.093724152617,
  3378.2242469204707,
  3380.418431034073,
  3390.012806840532,
  3398.1515495020453],
 'USA': [6752979530.172898,
  6534090367.760596,
  6854584791.815634,
  6719012891.651906,
  6658830817.767772,
  6901676664.092443],
 'USD_KRW': [1093.9051053067028,
  1092.5486239292582,
  1078.138626176035,
  1067.5014539359236,
  1072.373585207966,
  1086.2783413170582],
 'china': [12341723020.815351,
  12132232348.473543,
  12225030860.675817,
  12362068983.06136,
  12417211756.704187,
  12399742106.773584],
 'hongkong': [2607877591.4383435,
  2655222137.891178,
  3046825094.553913,
  2626122848.1815753,
  2661974524.5381393,
  3074466198.492547],
 'japan': [2293008996.6371803,
  

# LinearRegression

In [37]:
filename= '/content/drive/MyDrive/Export_Prediction-master/data/added_data.csv'
data = pd.read_csv(filename)
data = data.iloc[:-6,2:] # 202107-12 제거
data_train = data.iloc[:-6,:]
data_test = data.iloc[-6:,:] # 202101-06
X_train = data_train.iloc[:,1:]
X_test = pd.DataFrame(forcast_dict)
y_train = data_train.iloc[:,0]
y_test = data_test.iloc[:,0]

In [40]:
# standard scaling
scaler=StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

#LinearRegression
model=LinearRegression()
model.fit(X_train_std, y_train)
y_pred = model.predict(X_test_std)
rmse = round(metrics.mean_squared_error(y_test, y_pred,squared=False),3)

print('==LinearRegression==')
print(f'y_prediction : {y_pred}')
print(f'rmse : {rmse}')


#Lasso
model=Lasso(alpha=0.5)
model.fit(X_train_std, y_train)
y_pred = model.predict(X_test_std)
rmse = round(metrics.mean_squared_error(y_test, y_pred,squared=False),3)
print('==LassoRegression==')
print(f'y_prediction : {y_pred}')
print(f'rmse : {rmse}')

#Ridge
model=Ridge(alpha=0.5)
model.fit(X_train_std, y_train)
y_pred = model.predict(X_test_std)
rmse = round(metrics.mean_squared_error(y_test, y_pred,squared=False),3)
print('==RidgeRegression==')
print(f'y_prediction : {y_pred}')
print(f'rmse : {rmse}')

==LinearRegression==
y_prediction : [4.78651687e+10 4.66452497e+10 4.76499391e+10 4.73122810e+10
 4.76881295e+10 4.78633475e+10]
rmse : 4332749107.075
==LassoRegression==
y_prediction : [4.78651687e+10 4.66452497e+10 4.76499391e+10 4.73122810e+10
 4.76881295e+10 4.78633475e+10]
rmse : 4332749104.098
==RidgeRegression==
y_prediction : [4.78730893e+10 4.66644891e+10 4.77167934e+10 4.73210209e+10
 4.77045744e+10 4.79269969e+10]
rmse : 4298479536.689


TypeError: ignored